In [1]:
import os
import configparser
from datetime import timedelta

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now


curr_dir = os.getcwd()
main_folder = '\\'.join(curr_dir.split('\\')[:-1])

config = configparser.ConfigParser()
config.read(f'{main_folder}\config.ini')
TOKEN = config['API']['token']

In [2]:
# Посмотрим на объект свечи
with Client(TOKEN) as client:
        for candle in client.get_all_candles(  # это генератор, список свечей создать не получится
            figi="BBG004730N88",
            from_=now() - timedelta(days=365),
            interval=CandleInterval.CANDLE_INTERVAL_HOUR,
        ):
            example = candle.open.units + candle.open.nano * 1e-9
            print(candle)
            print(example)
            break
            

HistoricCandle(open=Quotation(units=318, nano=270000000), high=Quotation(units=320, nano=150000000), low=Quotation(units=314, nano=310000000), close=Quotation(units=315, nano=860000000), volume=1107947, time=datetime.datetime(2021, 5, 10, 7, 0, tzinfo=datetime.timezone.utc), is_complete=True)
318.27


- units -  целая часть суммы
- nano - дробная часть

In [3]:
# Получим все тикеты, выведем только российские
with Client(TOKEN) as client:
    i = 0
    for share in client.instruments.shares().instruments:
        if share.country_of_risk == 'RU':
            i += 1
            print(f'{i})\t{share.figi}\t{share.ticker}\t{share.name}')

1)	BBG00Y3XYV94	MDMG	Мать и дитя
2)	BBG00ZHCX1X2	FIXP	Fix Price Group
3)	BBG004730JJ5	MOEX	Московская Биржа
4)	BBG000VKG4R5	MRKU	МРСК Урала
5)	BBG000TJ6F42	MRKZ	Россети Северо-Запад
6)	BBG000Q7GG57	TGKB	ТГК-2
7)	BBG0029SFXB3	KZOS	ПАО «КАЗАНЬОРГСИНТЕЗ»
8)	BBG000CSZKR2	MBT	АДР МТС
9)	BBG005DXJS36	TCS	TCS Group (Tinkoff Bank holder)
10)	BBG006L8G4H1	YNDX	Yandex
11)	BBG004730ZJ9	VTBR	Банк ВТБ
12)	BBG000RMWQD4	ENPG	En+ Group
13)	BBG000RP8V70	CHMK	ЧМК
14)	BBG00475JZZ6	FEES	ФСК ЕЭС
15)	BBG000K3STR7	APTK	Аптечная сеть 36,6
16)	BBG0014PFYM2	RUGR	Русгрэйн
17)	BBG00KHGQ0H4	HHR	HeadHunter Group PLC
18)	BBG0100R9963	SGZH	Сегежа
19)	BBG000PKWCQ7	MRKV	МРСК Волги
20)	BBG004S68B31	ALRS	АЛРОСА
21)	BBG002458LF8	SELG	Селигдар
22)	BBG000NN8497	LKOD@GS	LUKOIL PJSC
23)	BBG000DBD6F6	KLSB	Калужская сбытовая компания
24)	BBG0019K04R5	LIFE	Фармсинтез
25)	BBG000R04X57	FLOT	Совкомфлот
26)	BBG00B8NN386	GRNT	Городские Инновационные Технологии
27)	BBG000F6YP24	ENRU	Энел Россия
28)	BBG00HY6V6H5	GTRK	ГТМ
29)	BBG003LYCM

In [4]:
# Получить стакан заявок на текущий момент
with Client(TOKEN) as client:
    order_book = client.market_data.get_order_book(depth=50, figi='BBG006L8G4H1')
order_book

GetOrderBookResponse(figi='BBG006L8G4H1', depth=50, bids=[], asks=[], last_price=Quotation(units=1688, nano=0), close_price=Quotation(units=1688, nano=0), limit_up=Quotation(units=0, nano=0), limit_down=Quotation(units=0, nano=0))

In [5]:
from tinkoff.invest import InstrumentIdType

In [6]:
# Получить информацию об инстументе/акции по FIGI
with Client(TOKEN) as client:
    instrument_info = client.instruments.get_instrument_by(
        id_type=InstrumentIdType.INSTRUMENT_ID_TYPE_FIGI,
        id='BBG006L8G4H1')
    share_info = client.instruments.share_by(
        id_type=InstrumentIdType.INSTRUMENT_ID_TYPE_FIGI,
        id='BBG006L8G4H1')

print(instrument_info.instrument, '\n')    
print(share_info.instrument)

Instrument(figi='BBG006L8G4H1', ticker='YNDX', class_code='TQBR', isin='NL0009805522', lot=1, currency='rub', klong=Quotation(units=2, nano=0), kshort=Quotation(units=2, nano=0), dlong=Quotation(units=0, nano=487500000), dshort=Quotation(units=0, nano=528400000), dlong_min=Quotation(units=0, nano=284100000), dshort_min=Quotation(units=0, nano=236300000), short_enabled_flag=False, name='Yandex', exchange='MOEX', country_of_risk='RU', country_of_risk_name='Российская Федерация', instrument_type='share', trading_status=<SecurityTradingStatus.SECURITY_TRADING_STATUS_BREAK_IN_TRADING: 4>, otc_flag=False, buy_available_flag=True, sell_available_flag=True, min_price_increment=Quotation(units=0, nano=200000000), api_trade_available_flag=True, uid='', real_exchange=<RealExchange.REAL_EXCHANGE_MOEX: 1>) 

Share(figi='BBG006L8G4H1', ticker='YNDX', class_code='TQBR', isin='NL0009805522', lot=1, currency='rub', klong=Quotation(units=2, nano=0), kshort=Quotation(units=2, nano=0), dlong=Quotation(uni

In [7]:
# Расписание торгов, вывод в библиотеке реализован плохо (строка хуйни)
with Client(TOKEN) as client:
    schedules = client.instruments.trading_schedules(
        exchange='MOEX',
        from_=now(),
        to=now() + timedelta(days=5))
schedules.exchanges

[TradingSchedule(exchange='MOEX', days=[TradingDay(date=datetime.datetime(2022, 5, 9, 0, 0, tzinfo=datetime.timezone.utc), is_trading_day=False, start_time=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), end_time=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), opening_auction_start_time=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), closing_auction_end_time=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), evening_opening_auction_start_time=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), evening_start_time=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), evening_end_time=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), clearing_start_time=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), clearing_end_time=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), premarket_start_time=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetim

In [8]:
# Лента сделок, исторические данные не выгружаются
with Client(TOKEN) as client:
    time_start = now() - timedelta(days=7) + timedelta(hours=15)
    last_trades = client.market_data.get_last_trades(
        figi='BBG006L8G4H1',
        from_=time_start,
        to=time_start + timedelta(hours=1),
    )
last_trades.trades

[]